In [ ]:
import sys

PATH_TO_WORKSPACE = '/Users/hassan/Documents/wod-prediction/WOD-prediction'
sys.path.append(PATH_TO_WORKSPACE)

from pipeline_objects.data_loader import DataLoader
from pipeline_objects.preprocessor import DataPreprocessor
from pipeline_objects.modeling import RandomForestModel


In [ ]:
loader = DataLoader(root_path = f'{PATH_TO_WORKSPACE}/Data',objects= ['open_results','descriptions','benchmark_stats'])
data = loader.load()

In [ ]:
# to test the pipeline faster, we can sample the data, remove this line to use all the data
data['open_results'] = data['open_results'].sample(10000)

In [ ]:
preprocessing_config = {
    'open_results':
    {
        'create_description_embeddings' : False,
    },
    # 'workout_descriptions':{},
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero'
    },
    # 'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessed_data = preprocessor.transform(data=data)


In [ ]:
rf_modeler = RandomForestModel(n_estimators = 100)
rf_modeler.fit(**preprocessed_data)

rf_modeler.show_results()